In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from PIL import Image
import time
import yaml
import numpy as np
from tqdm import tqdm
from torchvision import transforms
from torchvision.transforms import ToPILImage, ToTensor
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim import Adam
from torchvision.utils import save_image
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append("models")

import models
import utils
from utils import make_coord,set_save_path,ssim
import datasets
from test import eval_psnr,batched_predict

from models import losses
from models.liif import LIIF
from models.discriminator import Discriminator
from models.losses import AdversarialLoss
%matplotlib inline

In [8]:
class EdgeConv(nn.Module):
    def __init__(self):
        super().__init__()
        k = torch.Tensor([[.05, .25, .4, .25, .05]])
        self.kernel = torch.matmul(k.t(), k).unsqueeze(0).repeat(3, 1, 1, 1)
        if torch.cuda.is_available():
            self.kernel = self.kernel.cuda()


    def conv_gauss(self, img):
        n_channels, _, kw, kh = self.kernel.shape
        img = F.pad(img, (kw//2, kh//2, kw//2, kh//2), mode='replicate')
        return F.conv2d(img, self.kernel, groups=n_channels)

    def laplacian_kernel(self, current):
        filtered = self.conv_gauss(current)     # filter
        down = filtered[:, :, ::2, ::2]         # downsample
        new_filter = torch.zeros_like(filtered)
        new_filter[:, :, ::2, ::2] = down*4     # upsample
        filtered = self.conv_gauss(new_filter)  # filter
        diff = current - filtered
        return diff

In [14]:
edge =EdgeConv().cuda()

In [20]:
img = Image.open(r'load\div2k\DIV2K_valid_LR_bicubic\X2\0802x2.png')
#img = transforms.Resize((int(img.height/2),int(img.width/2)),Image.BICUBIC)(img)
timg = transforms.ToTensor()(img) #[3,LR_H,LR_W]
edgemap = edge.laplacian_kernel(timg.unsqueeze(0).cuda())
transforms.ToPILImage()(edgemap[0]).save('0802x2edge.jpg')

In [2]:
import csv

In [16]:
dic = {}
with open(r"testimg\WHURS19-edsrblx2\WHURS19-edsrblx2.csv", "r") as f:
    f_csv = csv.reader(f)
    for line in f_csv:
        img = line[0]
        psnr = float(line[1])
        ssim = float(line[2])
        dic[f"{img}"] = (psnr,ssim)



In [17]:
dic

{'railwayStation_46.jpg': (25.4852, 0.8672),
 'commercial_49.jpg': (26.4611, 0.8977),
 'residential_50.jpg': (26.496, 0.8933),
 'commercial_48.jpg': (26.861, 0.8924),
 'railwayStation_48.jpg': (26.9273, 0.895),
 'port_49.jpg': (26.9783, 0.8608),
 'residential_49.jpg': (27.2518, 0.902),
 'residential_46.jpg': (27.518, 0.9201),
 'residential_47.jpg': (27.5188, 0.9093),
 'port_47.jpg': (27.7927, 0.9194),
 'Mountain_44.jpg': (27.8186, 0.8484),
 'Mountain_42.jpg': (27.8919, 0.8767),
 'residential_45.jpg': (27.9138, 0.9165),
 'residential_48.jpg': (27.9473, 0.891),
 'railwayStation_47.jpg': (28.0, 0.907),
 'Mountain_50.jpg': (28.0622, 0.8542),
 'railwayStation_49.jpg': (28.0946, 0.8957),
 'forest_42.jpg': (28.0975, 0.8457),
 'commercial_47.jpg': (28.1056, 0.8919),
 'industrial_50.jpg': (28.2222, 0.8935),
 'railwayStation_44.jpg': (28.361, 0.8605),
 'Mountain_47.jpg': (28.5159, 0.8446),
 'Park_45.jpg': (28.588, 0.9201),
 'commercial_46.jpg': (28.605, 0.9183),
 'industrial_46.jpg': (28.6548, 0